<a href="https://colab.research.google.com/github/CaloCare/MachineLearning/blob/main/CaloCare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**CaloCare**

## Menentukan Pertanyaan Bisnis

## Import Semua Packages/Library yang Digunakan

In [1]:
!pip install kaggle

In [2]:
! mkdir ~/.kaggle

In [6]:
!cp kaggle.json ~/.kaggle

In [7]:
!kaggle datasets download -d 'rusqi29/food-detection-dataset-for-calorie-measurement'

Dataset URL: https://www.kaggle.com/datasets/rusqi29/food-detection-dataset-for-calorie-measurement
License(s): other
... resuming from 127926272 bytes (4872669093 bytes left) ...
100% 4.66G/4.66G [03:46<00:00, 14.3MB/s]
100% 4.66G/4.66G [03:46<00:00, 21.5MB/s]


In [8]:
import zipfile
import os

# Ganti path_zip dengan path
path_zip = "/content/food-detection-dataset-for-calorie-measurement.zip"

# Ganti path_extract dengan path tempat Anda ingin mengekstrak file zip
path_extract = "/content/FooDD"

# Mengekstrak file zip
with zipfile.ZipFile(path_zip, 'r') as zip_ref:
    zip_ref.extractall(path_extract)

# Menampilkan daftar file yang telah diekstrak
extracted_files = os.listdir(path_extract)
print("File yang telah diekstrak:", extracted_files)

File yang telah diekstrak: ['FooDD']


In [9]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [17]:
path_dataset = "/content/FooDD/FooDD/FooDD"

# Tentukan ukuran batch
batch_size = 128

# Membuat generator untuk data training
train_datagen = ImageDataGenerator(
    validation_split=0.2,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2
)

train_generator = train_datagen.flow_from_directory(
    path_dataset,
    target_size=(224, 224),
    batch_size=batch_size,
    seed=100,
    class_mode='categorical',
    subset='training'
)

# Membuat generator untuk data validasi dan test (gabungan)
validation_test_generator = train_datagen.flow_from_directory(
    path_dataset,
    target_size=(224, 224),
    batch_size=batch_size,
    seed=200,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

# Hitung jumlah data validasi dan test
num_validation_test = validation_test_generator.samples
num_test = int(num_validation_test * 0.5)  # 50% for test, adjust as needed
num_validation = num_validation_test - num_test

# **Instead of slicing, iterate to split data into validation and test sets**
validation_data = []
validation_labels = []
for i in range(num_validation // batch_size + (num_validation % batch_size > 0)):  # Iterate over batches, handle the remainder
    batch_x, batch_y = next(validation_test_generator) # **Use next(validation_test_generator)** to get the next batch
    validation_data.append(batch_x)
    validation_labels.append(batch_y)

test_data = []
test_labels = []

# Reset the generator before iterating for test data
validation_test_generator.reset()
validation_test_generator.batch_index = num_validation // batch_size  # Start from the next batch

for i in range(num_test // batch_size + (num_test % batch_size > 0)):  # Iterate over batches, handle the remainder
    batch_x, batch_y = next(validation_test_generator) # **Use next(validation_test_generator)** to get the next batch
    test_data.append(batch_x)
    test_labels.append(batch_y)


validation_data = np.concatenate(validation_data)
validation_labels = np.concatenate(validation_labels)
test_data = np.concatenate(test_data)
test_labels = np.concatenate(test_labels)

# Output informasi tentang dataset
print("Jumlah kelas:", len(train_generator.class_indices))
print("Jumlah data training:", train_generator.n)
print("Jumlah data validation:", num_validation)  # Use calculated num_validation
print("Jumlah data test:", num_test)  # Use calculated num_test

Found 3117 images belonging to 20 classes.
Found 768 images belonging to 20 classes.
Jumlah kelas: 20
Jumlah data training: 3117
Jumlah data validation: 384
Jumlah data test: 384


## Data Wrangling

### Gathering Data

**Insight:**
- xxx
- xxx

### Assessing Data

**Insight:**
- xxx
- xxx

### Cleaning Data

**Insight:**
- xxx
- xxx

## Exploratory Data Analysis (EDA)

### Explore ...

**Insight:**
- xxx
- xxx

## Visualization & Explanatory Analysis

### Pertanyaan 1:

### Pertanyaan 2:

**Insight:**
- xxx
- xxx

## Analisis Lanjutan (Opsional)

## Conclusion

- Conclution pertanyaan 1
- Conclution pertanyaan 2